# Handling Missing Data

In this section, we will study ways to identify and treat missing data. We will:
- Identify missing data in dataframes
- Treat (delete or impute) missing values

There are various reasons for missing data, such as, human-errors during data-entry, non availability at the end of the user (e.g. DOB of certain people), etc. Most often, the reasons are simply unknown.

In python, missing data is represented using either of the two objects ```NaN``` (Not a Number) or ```NULL```. We'll not get into the differences between them and how Python stores them internally etc. We'll focus on studying ways to identify and treat missing values in Pandas dataframes.

There are four main methods to identify and treat missing data:
- ```isnull()```: Indicates presence of missing values, returns a boolean
- ```notnull()```: Opposite of ```isnull()```, returns a boolean
- ```dropna()```: Drops the missing values from a data frame and returns the rest
- ```fillna()```: Fills (or imputes) the missing values by a specified value


For this exercise, we will use the **Melbourne house pricing dataset**. 


In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv("melbourne.csv")
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,03-09-2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,03-12-2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,04-02-2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,04-02-2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,04-03-2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0


The first few rows contain missing values, represented as NaN.

Let's quickly look at the structure of the data frame, types of columns, etc.

In [2]:
# approx 23k rows, 21 columns
print(df.shape)
print(df.info())

(23547, 21)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23547 entries, 0 to 23546
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         23547 non-null  object 
 1   Address        23547 non-null  object 
 2   Rooms          23547 non-null  int64  
 3   Type           23547 non-null  object 
 4   Price          18396 non-null  float64
 5   Method         23547 non-null  object 
 6   SellerG        23547 non-null  object 
 7   Date           23547 non-null  object 
 8   Distance       23546 non-null  float64
 9   Postcode       23546 non-null  float64
 10  Bedroom2       19066 non-null  float64
 11  Bathroom       19063 non-null  float64
 12  Car            18921 non-null  float64
 13  Landsize       17410 non-null  float64
 14  BuildingArea   10018 non-null  float64
 15  YearBuilt      11540 non-null  float64
 16  CouncilArea    15656 non-null  object 
 17  Lattitude      19243 non-null  float64

## Identifying Missing Values

The methods ```isnull()``` and ```notnull()``` are the most common ways of identifying missing values. 

While handling missing data, you first need to identify the rows and columns containing missing values, count the number of missing values, and then decide how you want to treat them.

It is important that **you treat missing values in each column separately**, rather than implementing a single solution (e.g. replacing NaNs by the mean of a column) for all columns.

```isnull()``` returns a boolean (True/False) which can then be used to find the rows or columns containing missing values.

In [3]:
# isnull()
df.isnull()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,False,False,False,False,True,False,False,False,False,False,...,False,False,False,True,True,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,True,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False,...,False,False,False,True,True,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23542,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
23543,False,False,False,False,True,False,False,False,False,False,...,True,True,True,True,True,True,False,False,False,False
23544,False,False,False,False,False,False,False,False,False,False,...,False,False,True,True,True,True,False,False,False,False
23545,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False


###  Identifying Missing Values in Columns
Let's first compute the total number of missing values in the data frame. You can calculate the number of missing values in each column by ```df.isnull().sum()``` 

In [4]:
# summing up the missing values (column-wise)
df.isnull().sum()

Suburb               0
Address              0
Rooms                0
Type                 0
Price             5151
Method               0
SellerG              0
Date                 0
Distance             1
Postcode             1
Bedroom2          4481
Bathroom          4484
Car               4626
Landsize          6137
BuildingArea     13529
YearBuilt        12007
CouncilArea       7891
Lattitude         4304
Longtitude        4304
Regionname           1
Propertycount        1
dtype: int64

Note that some columns have extremely **large number of missing values**, such as Price, Bedroom2, Bathroom, BuildingArea, YearBuilt etc. In such cases, one should be careful in handling missing values, since if you replace them by arbitrary numbers such as mean, median etc., the entire further analysis may throw unrealistic or unexpected results.

The functions ```any()``` and ```all()``` are quite useful to identify rows and columns having missing values:
- ```any()``` returns ```True``` when at least one value satisfies a condition (equivalent to logical ```or```)
- ```all()``` returns ```True``` when all the values satisfy a condition (equivalent to logical ```and```)

In [4]:
# columns having at least one missing value
df.isnull().any()

# above is equivalent to axis=0 (by default, any() operates on columns)
df.isnull().any(axis=0)

Suburb           False
Address          False
Rooms            False
Type             False
Price             True
Method           False
SellerG          False
Date             False
Distance          True
Postcode          True
Bedroom2          True
Bathroom          True
Car               True
Landsize          True
BuildingArea      True
YearBuilt         True
CouncilArea       True
Lattitude         True
Longtitude        True
Regionname        True
Propertycount     True
dtype: bool

We have identified columns having missing values and have computed the number of missing values in each. Let's do the same for rows.

### Identifying  Missing Values in Rows

The methods ```any()``` and ```all()``` can be used to identify rows having **at least one** and **all** missing values respectively. To specify that the operation should be done on rows, you need to use ```axis=1``` as an argument.

In [5]:
# rows having at least one missing value
df.isnull().any(axis=1)

0         True
1         True
2        False
3         True
4        False
         ...  
23542     True
23543     True
23544     True
23545     True
23546     True
Length: 23547, dtype: bool

In [6]:
# rows having all missing values
df.isnull().all(axis=1)

0        False
1        False
2        False
3        False
4        False
         ...  
23542    False
23543    False
23544    False
23545    False
23546    False
Length: 23547, dtype: bool

In [8]:
# sum it up to check how many rows have all missing values
df.isnull().all(axis=1).sum()

0

Thus, there are no rows having all missing values (we'd remove them if there were any). 

Often, you may also want to remove the rows having more than a certain threshold number of missing values. To do that, you need to count the number of missing values in each row using ```sum()```.

In [7]:
# sum of misisng values in each row
df.isnull().sum(axis=1)

0        3
1        2
2        0
3        3
4        0
        ..
23542    2
23543    8
23544    4
23545    1
23546    2
Length: 23547, dtype: int64

We have now identified:
- The number of missing values in columns
- The number of missing values in rows

Let's now move ahead and treat the missing values.

### Treating Missing Values

There are broadly two ways to treat missing values:
1. Delete: Delete the missing values 
2. Impute: 
    - Imputing by a simple statistic: Replace the missing values by another value, commonly the mean, median, mode etc. 
    - Predictive techniques: Use statistical models such as k-NN, SVM etc. to predict and impute missing values
   

In general, imputation makes assumptions about the missing values and replaces missing values by arbitrary numbers such as mean, median etc. It should be used only when you are reasonably confident about the assumptions.

Otherwise, deletion is often safer and recommended. You may lose some data, but will not make any unreasonable assumptions.

**Caution**: Always have a backup of the original data if you're deleting missing values.  

<hr>
**Additional Stuff for Nerds**

How you treat missing values should ideally depend upon an understnading of why missing values occur. The reasons are classified into categories such as *missing completely at random, missing at random, misisngness that depends on the missing value itself etc.* 
 
 
We'll not discuss *why missing values occur*, though you can read this article if interested: http://www.stat.columbia.edu/~gelman/arm/missing.pdf
<hr>

### Treating Missing Values in Columns

Let's now treat missing values in columns. Let's look at the number of NaNs in each column again, this time as the *percentage of missing values in each column*. Notice that we calculate the number of rows as ```len(df.index)```.

In [8]:
# summing up the missing values (column-wise)
round(100*(df.isnull().sum()/len(df.index)), 2)

Suburb            0.00
Address           0.00
Rooms             0.00
Type              0.00
Price            21.88
Method            0.00
SellerG           0.00
Date              0.00
Distance          0.00
Postcode          0.00
Bedroom2         19.03
Bathroom         19.04
Car              19.65
Landsize         26.06
BuildingArea     57.46
YearBuilt        50.99
CouncilArea      33.51
Lattitude        18.28
Longtitude       18.28
Regionname        0.00
Propertycount     0.00
dtype: float64

Notice that there are columns having almost 22%, 19%, 26%, 57% etc. missing values. When dealing with columns, you have two simple choices - either **delete or retain the column.** If you retain the column, you'll have to treat (i.e. delete or impute) the rows having missing values.

If you delete the missing rows, you lose data. If you impute, you introduce bias.

Apart from the number of missing values, the decision to delete or retain a variable depends on various other factors, such as:
- the analysis task at hand
- the usefulness of the variable (based on your understanding of the problem)
- the total size of available data (if you have enough, you can afford to throw away some of it)
- etc.

For e.g. let's say that we want to build a (linear regression) model to predict the house prices in Melbourne. Now, even though the variable ```Price``` has about 22% missing values, you cannot drop the variable, since that is what you want to predict. 

Similarly, you would expect some other variables such as ```Bedroom2```, ```Bathroom```, ```Landsize``` etc. to be important predictors of ```Price```, and thus cannot remove those columns.

There are others such as ```BuildingArea```, which although seem important, have more than 50% missing values. It is impossible to either delete or impute the rows corresponding to such large number of missing values without losing a lot of data or introducing heavy bias. 



Thus, for this exercise, let's remove the columns having more than 30% missing values, i.e. ```BuildingArea```, ```YearBuilt```, ```CouncilArea```.



In [9]:
# removing the three columns
df = df.drop('BuildingArea', axis=1)
df = df.drop('YearBuilt', axis=1)
df = df.drop('CouncilArea', axis=1)

round(100*(df.isnull().sum()/len(df.index)), 2)

Suburb            0.00
Address           0.00
Rooms             0.00
Type              0.00
Price            21.88
Method            0.00
SellerG           0.00
Date              0.00
Distance          0.00
Postcode          0.00
Bedroom2         19.03
Bathroom         19.04
Car              19.65
Landsize         26.06
Lattitude        18.28
Longtitude       18.28
Regionname        0.00
Propertycount     0.00
dtype: float64

We now have columns having maximum 26% missing values (```Landsize```). Next, we need to treat the rows.

### Treating Missing Values in Rows

Now, we need to either delete or impute the missing values. First, let's see if there are any rows having a significant number of missing values. If so, we can drop those rows, and then take a decision to delete or impute the rest.

After dropping three columns, we now have 18 columns to work with. Just to inspect rows with missing values, let's have a look at the rows having more than 5 missing values.

In [10]:
df[df.isnull().sum(axis=1) > 5]

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,Lattitude,Longtitude,Regionname,Propertycount
15,Abbotsford,217 Langridge St,3,h,1000000.0,S,Jellis,08-10-2016,2.5,3067.0,NaN,NaN,NaN,NaN,NaN,NaN,Northern Metropolitan,4019.0
16,Abbotsford,18a Mollison St,2,t,745000.0,S,Jellis,08-10-2016,2.5,3067.0,NaN,NaN,NaN,NaN,NaN,NaN,Northern Metropolitan,4019.0
19,Abbotsford,403/609 Victoria St,2,u,542000.0,S,Dingle,08-10-2016,2.5,3067.0,NaN,NaN,NaN,NaN,NaN,NaN,Northern Metropolitan,4019.0
21,Abbotsford,25/84 Trenerry Cr,2,u,760000.0,SP,Biggin,10-12-2016,2.5,3067.0,NaN,NaN,NaN,NaN,NaN,NaN,Northern Metropolitan,4019.0
22,Abbotsford,106/119 Turner St,1,u,481000.0,SP,Purplebricks,10-12-2016,2.5,3067.0,NaN,NaN,NaN,NaN,NaN,NaN,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23495,Templestowe Lower,1/207 Manningham Rd,2,u,550000.0,PI,Barry,26-08-2017,12.4,3107.0,NaN,NaN,NaN,NaN,NaN,NaN,Eastern Metropolitan,5420.0
23499,Thornbury,1/128 Dundas St,2,t,770000.0,PI,McGrath,26-08-2017,7.0,3071.0,NaN,NaN,NaN,NaN,NaN,NaN,Northern Metropolitan,8870.0
23502,Thornbury,111 Pender St,2,t,858000.0,S,Jellis,26-08-2017,7.0,3071.0,NaN,NaN,NaN,NaN,NaN,NaN,Northern Metropolitan,8870.0
23508,Toorak,21/1059 Malvern Rd,2,u,720000.0,VB,Beller,26-08-2017,4.1,3142.0,NaN,NaN,NaN,NaN,NaN,NaN,Southern Metropolitan,7217.0


Notice an interesting pattern - many rows have multiple columns missing. Since each row represents a house, it indicates that there are houses (observations) whose majority data has either not been collected or is unavailable. Such observations are anyway unlikely to contribute to prediction of prices. 

Thus we can remove the rows with (say) more than 5 missing values.

In [13]:
# count the number of rows having > 5 missing values
# use len(df.index)
len(df[df.isnull().sum(axis=1) > 5].index)

4278

In [14]:
# 4278 rows have more than 5 missing values
# calculate the percentage
100*(len(df[df.isnull().sum(axis=1) > 5].index) / len(df.index))

18.16791948018856

Thus, about 18% rows have more than 5 missing values. Let's remove these rows and count the number of missing values remaining.

In [15]:
# retaining the rows having <= 5 NaNs
df = df[df.isnull().sum(axis=1) <= 5]

# look at the summary again
round(100*(df.isnull().sum()/len(df.index)), 2)

Suburb            0.00
Address           0.00
Rooms             0.00
Type              0.00
Price            21.71
Method            0.00
SellerG           0.00
Date              0.00
Distance          0.00
Postcode          0.00
Bedroom2          1.05
Bathroom          1.07
Car               1.81
Landsize          9.65
Lattitude         0.13
Longtitude        0.13
Regionname        0.00
Propertycount     0.00
dtype: float64

Notice that now, we have removed most of the rows where multiple columns (```Bedroom2```, ```Bathroom```, ```Landsize```) were missing. 

Now, we still have about 21% missing values in the column ```Price``` and 9% in ```Landsize```. Since ```Price``` still contains a lot of missing data (and imputing 21% values of a variable you want to predict will introduce heavy bias), its a bad idea to impute those values. 

Thus, let's remove the missing rows in ```Price``` as well. Notice that you can use ```np.isnan(df['column'])``` to filter out the corresonding rows, and use a ```~``` to discard the values satisfying the condition.

In [16]:
# removing NaN Price rows
df = df[~np.isnan(df['Price'])]

round(100*(df.isnull().sum()/len(df.index)), 2)


Suburb           0.00
Address          0.00
Rooms            0.00
Type             0.00
Price            0.00
Method           0.00
SellerG          0.00
Date             0.00
Distance         0.00
Postcode         0.00
Bedroom2         1.05
Bathroom         1.07
Car              1.76
Landsize         9.83
Lattitude        0.15
Longtitude       0.15
Regionname       0.00
Propertycount    0.00
dtype: float64

Now, you have ```Landsize``` as the only variable having a significant number of missing values. Let's give this variable a chance and consider imputing the NaNs. 

The decision (whether and how to impute) will depend upon the distribution of the variable. For e.g., if the variable is such that all the observations lie in a short range (say between 800 sq. ft to 820 sq.ft), you can take a call to impute the missing values by something like the mean or median ```Landsize```. 

Let's look at the distribution.

In [17]:
df['Landsize'].describe()

count     13603.000000
mean        558.116371
std        3987.326586
min           0.000000
25%         176.500000
50%         440.000000
75%         651.000000
max      433014.000000
Name: Landsize, dtype: float64

Notice that the minimum is 0, max is 433014, the mean is 558 and median (50%) is 440. There's a significant variation in the 25th and the 75th percentile as well (176 to 651). 

Thus, imputing this with mean/median seems quite biased, and so we should remove the NaNs.

In [18]:
# removing NaNs in Landsize
df = df[~np.isnan(df['Landsize'])]

round(100*(df.isnull().sum()/len(df.index)), 2)

Suburb           0.00
Address          0.00
Rooms            0.00
Type             0.00
Price            0.00
Method           0.00
SellerG          0.00
Date             0.00
Distance         0.00
Postcode         0.00
Bedroom2         0.00
Bathroom         0.01
Car              0.46
Landsize         0.00
Lattitude        0.16
Longtitude       0.16
Regionname       0.00
Propertycount    0.00
dtype: float64

We have reduced the NaNs significantly now. Only the variables ```Bathroom```, ```Car```, ```Lattitude``` and ```Longitude``` have a small number of missing values (most likely, the same rows will have ```Lattitude``` and ```Longitude``` missing).

Let's first look at ```Lattitude``` and ```Longitude```.

In [19]:
# rows having Lattitude and Longitude missing
df[np.isnan(df['Lattitude'])]

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,Lattitude,Longtitude,Regionname,Propertycount
2572,Burwood,23 Monica St,3,h,990000.0,VB,Fletchers,17-09-2016,11.7,3125.0,3.0,2.0,2.0,263.0,NaN,NaN,Southern Metropolitan,5678.0
3257,Clifton Hill,3/268 Alexandra Pde E,1,u,363000.0,S,hockingstuart,27-06-2016,3.4,3068.0,1.0,1.0,1.0,0.0,NaN,NaN,Northern Metropolitan,2954.0
4485,Footscray,483 Barkly St,3,t,781000.0,S,Jas,27-11-2016,6.4,3011.0,3.0,2.0,2.0,98.0,NaN,NaN,Western Metropolitan,7570.0
5170,Hampton East,7 Seafoam St,4,t,1185000.0,S,RT,28-05-2016,14.5,3188.0,4.0,3.0,1.0,300.0,NaN,NaN,Southern Metropolitan,2356.0
10745,Williamstown North,4/9 Adeline St,1,u,355000.0,S,Sweeney,27-11-2016,8.9,3016.0,1.0,1.0,1.0,46.0,NaN,NaN,Western Metropolitan,802.0
13223,Melbourne,1913/228 Abeckett St,3,u,1175000.0,PI,Icon,29-04-2017,2.8,3000.0,3.0,3.0,2.0,0.0,NaN,NaN,Northern Metropolitan,17496.0
14008,Brooklyn,9 Richards Ct,3,h,750000.0,S,hockingstuart,20-05-2017,10.9,3012.0,3.0,1.0,2.0,667.0,NaN,NaN,Western Metropolitan,962.0
14132,North Melbourne,13/201 Abbotsford St,2,t,755000.0,PI,Nelson,29-04-2017,2.3,3051.0,2.0,1.0,1.0,1537.0,NaN,NaN,Northern Metropolitan,6821.0
14139,Oakleigh South,4 Druitt St,4,h,1205500.0,S,Woodards,22-04-2017,14.7,3167.0,4.0,2.0,2.0,553.0,NaN,NaN,South-Eastern Metropolitan,3692.0
14142,Oakleigh South,298 Warrigal Rd,3,h,799999.0,S,Woodards,29-04-2017,14.7,3167.0,3.0,2.0,4.0,590.0,NaN,NaN,South-Eastern Metropolitan,3692.0


As expected, the same rows have ```Lattitude``` and ```Longitude``` missing. Let's look at the summary stats of these.

In [20]:
df.loc[:, ['Lattitude', 'Longtitude']].describe()

,Lattitude,Longtitude
count,13581.000000,13581.000000
mean,-37.809204,144.995221
std,0.079257,0.103913
min,-38.182550,144.431810
25%,-37.856820,144.929600
50%,-37.802360,145.000100
75%,-37.756400,145.058320
max,-37.408530,145.526350


Notice that the distribution of both ```Lattitude``` and ```Longitude``` is quite narrow. 

A good way to estimate the 'spread of data' is to look at the difference between the mean and the median (lower the better), and the variation from 25th to 75th percentile (quite small in this case).

Thus, let's impute the missing values by the mean value of ```Lattitude``` and ```Longitude``` respectively.

In [21]:
# imputing Lattitude and Longitude by mean values
df.loc[np.isnan(df['Lattitude']), ['Lattitude']] = df['Lattitude'].mean()
df.loc[np.isnan(df['Longtitude']), ['Longtitude']] = df['Longtitude'].mean()

round(100*(df.isnull().sum()/len(df.index)), 2)

Suburb           0.00
Address          0.00
Rooms            0.00
Type             0.00
Price            0.00
Method           0.00
SellerG          0.00
Date             0.00
Distance         0.00
Postcode         0.00
Bedroom2         0.00
Bathroom         0.01
Car              0.46
Landsize         0.00
Lattitude        0.00
Longtitude       0.00
Regionname       0.00
Propertycount    0.00
dtype: float64

We now have ```Bathroom``` and ```Car``` with 0.01% and 0.46% NaNs respectively. 

Since these are very small number of rows, it does not really matter whether you delete or impute. However, let's have a look at the distributions.

In [22]:
df.loc[:, ['Bathroom', 'Car']].describe()

,Bathroom,Car
count,13602.000000,13540.000000
mean,1.534921,1.610414
std,0.691834,0.962244
min,0.000000,0.000000
25%,1.000000,1.000000
50%,1.000000,2.000000
75%,2.000000,2.000000
max,8.000000,10.000000


These two are integer type variables, and thus have values 0, 1, 2 etc. You cannot impute the NaNs by the mean or the median (1.53 bathrooms does not make sense!).

Thus, you need to impute them by the mode - the most common occurring value.



In [23]:
# converting to type 'category'
df['Car'] = df['Car'].astype('category')

# displaying frequencies of each category
df['Car'].value_counts()

2.0     5606
1.0     5515
0.0     1026
3.0      748
4.0      507
5.0       63
6.0       54
8.0        9
7.0        8
10.0       3
9.0        1
Name: Car, dtype: int64

The most common value of ```Car``` is 2 (dtype is int), so let's impute the NaNs by that.

In [24]:
# imputing NaNs by 2.0
df.loc[pd.isnull(df['Car']), ['Car']] = 2
round(100*(df.isnull().sum()/len(df.index)), 2)

Suburb           0.00
Address          0.00
Rooms            0.00
Type             0.00
Price            0.00
Method           0.00
SellerG          0.00
Date             0.00
Distance         0.00
Postcode         0.00
Bedroom2         0.00
Bathroom         0.01
Car              0.00
Landsize         0.00
Lattitude        0.00
Longtitude       0.00
Regionname       0.00
Propertycount    0.00
dtype: float64

Similarly for ```Bathroom```:

In [25]:
# converting to type 'category'
df['Bathroom'] = df['Bathroom'].astype('category')

# displaying frequencies of each category
df['Bathroom'].value_counts()

1.0    7517
2.0    4987
3.0     921
4.0     106
0.0      34
5.0      28
6.0       5
8.0       2
7.0       2
Name: Bathroom, dtype: int64

In [26]:
# imputing NaNs by 1
df.loc[pd.isnull(df['Bathroom']), ['Bathroom']] = 1
round(100*(df.isnull().sum()/len(df.index)), 2)

Suburb           0.0
Address          0.0
Rooms            0.0
Type             0.0
Price            0.0
Method           0.0
SellerG          0.0
Date             0.0
Distance         0.0
Postcode         0.0
Bedroom2         0.0
Bathroom         0.0
Car              0.0
Landsize         0.0
Lattitude        0.0
Longtitude       0.0
Regionname       0.0
Propertycount    0.0
dtype: float64

We now have a dataframe with no missing values. Let's finally look at how many rows (apart from three columns) we have lost in the process (originally we had 23547):

In [27]:
df.shape

(13603, 18)

In [28]:
# fraction of rows lost
len(df.index)/23547

0.5776956724848176

Thus, we have lost about 42% observations in cleaning the missing values. 